In [1]:
import pandas as pd 
import jsonlines 

In [ ]:
# input_file = "tmp/adult.score.jsonl"
# input_file = "tmp/diabetes.score.jsonl"
input_file = "syned_datasets/syn1/switzerland.score.jsonl"
with jsonlines.open(input_file) as reader:
    data = list(reader)

In [3]:
df = pd.DataFrame(data)
df['status_score'] = df['status_score'].fillna(0)
df['gain_score'] = df['gain_score'].fillna(0)

In [4]:
sucs_df = df.groupby('syn_pmt_id').filter(lambda x: (x['status_score'] == 5).any())
# sucs_df = df.groupby('syn_pmt_id').filter(lambda x: (x['gain_score'] > 0).any())

In [5]:
from itertools import combinations
def generate_index_combinations(group):
    indices = group.index.tolist()
    return list(combinations(indices, 2))

def prefer_r1_than_r2(row1, row2):
    if row1['status_score']==5 and row2['status_score']==5:
        return row1['gain_score']>row2['gain_score']
    else:
        return row1['status_score'] > row2['status_score']
    
def reformat_data(row):
    js = []
    js.append({"content": row['prompt'], "role": "user"})
    js.append({"content": row['answer'], "role": "assistant"})
    return js



In [6]:
# Group by 'syn_pmt_id' and get the list of index combinations
index_combinations = sucs_df.groupby('syn_pmt_id').apply(
    lambda group: generate_index_combinations(group)
).explode().tolist()

prefer_dataset = []
for idx1, idx2 in index_combinations:
    row1 = df.iloc[idx1]
    row2 = df.iloc[idx2]
    if prefer_r1_than_r2(row1, row2):
        chosen = row1
        reject = row2
    else:
        chosen = row2
        reject = row1
    chosen_socre = chosen['gain_score'] + chosen['status_score']
    reject_score = reject['gain_score'] + reject['status_score']
    if chosen_socre == reject_score:
        continue
    chosen = reformat_data(chosen)
    reject = reformat_data(reject)
    prefer_dataset.append({"chosen": chosen, "reject": reject,
                          "chosen_score": chosen_socre, "reject_score": reject_score})
    
    

/tmp/ipykernel_2671716/319028270.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  index_combinations = sucs_df.groupby('syn_pmt_id').apply(


In [7]:
prefer_df = pd.DataFrame(prefer_dataset)
from datasets import Dataset
dataset = Dataset.from_pandas(prefer_df)
# dataset.save_to_disk("syned_datasets/syn1_dpo/adult")
# dataset.save_to_disk("syned_datasets/syn1_dpo/diabetes")
dataset.save_to_disk("syned_datasets/syn1_dpo/german")

Saving the dataset (0/1 shards):   0%|          | 0/2122 [00:00<?, ? examples/s]

In [12]:
dataset[0]

{'chosen': [{'content': 'Here are five entries of tabular data in JSON format, each consisting of twenty-one features. Each feature is described in a structured JSON format: "feature name": "value" . The target feature status is a classification task.\n\nsample one:\n{\n\t"Status of existing checking account": a14,\n\t"Duration in month": 48,\n\t"Credit history": a31,\n\t"Purpose": a49,\n\t"Credit amount": 3609.0000,\n\t"Savings account/bonds": a61,\n\t"Present employment since": a73,\n\t"Installment rate in percentage of disposable income": 1,\n\t"Personal status and sex": a92,\n\t"Other debtors / guarantors": a101,\n\t"Present residence since": 1,\n\t"Property": a121,\n\t"Age in years": 27,\n\t"Other installment plans": a142,\n\t"Housing": a152,\n\t"Number of existing credits at this bank": 1,\n\t"Job": a173,\n\t"Number of people being liable to provide maintenance for": 1,\n\t"Telephone": a191,\n\t"foreign worker": a201,\n\t"status": 0\n}\n\nsample two:\n{\n\t"Status of existing che

In [9]:
[{'content': 'Here are five entries of tabular data in JSON format, each consisting of twenty-one features. Each feature is described in a structured JSON format: "feature name": "value" . The target feature status is a classification task.\n\nsample one:\n{\n\t"Status of existing checking account": a14,\n\t"Duration in month": 48,\n\t"Credit history": a31,\n\t"Purpose": a49,\n\t"Credit amount": 3609.0000,\n\t"Savings account/bonds": a61,\n\t"Present employment since": a73,\n\t"Installment rate in percentage of disposable income": 1,\n\t"Personal status and sex": a92,\n\t"Other debtors / guarantors": a101,\n\t"Present residence since": 1,\n\t"Property": a121,\n\t"Age in years": 27,\n\t"Other installment plans": a142,\n\t"Housing": a152,\n\t"Number of existing credits at this bank": 1,\n\t"Job": a173,\n\t"Number of people being liable to provide maintenance for": 1,\n\t"Telephone": a191,\n\t"foreign worker": a201,\n\t"status": 0\n}\n\nsample two:\n{\n\t"Status of existing checking account": a12,\n\t"Duration in month": 48,\n\t"Credit history": a30,\n\t"Purpose": a40,\n\t"Credit amount": 8358.0000,\n\t"Savings account/bonds": a63,\n\t"Present employment since": a72,\n\t"Installment rate in percentage of disposable income": 1,\n\t"Personal status and sex": a92,\n\t"Other debtors / guarantors": a101,\n\t"Present residence since": 1,\n\t"Property": a123,\n\t"Age in years": 30,\n\t"Other installment plans": a143,\n\t"Housing": a152,\n\t"Number of existing credits at this bank": 2,\n\t"Job": a173,\n\t"Number of people being liable to provide maintenance for": 1,\n\t"Telephone": a191,\n\t"foreign worker": a201,\n\t"status": 0\n}\n\nsample three:\n{\n\t"Status of existing checking account": a12,\n\t"Duration in month": 60,\n\t"Credit history": a33,\n\t"Purpose": a43,\n\t"Credit amount": 7418.0000,\n\t"Savings account/bonds": a65,\n\t"Present employment since": a73,\n\t"Installment rate in percentage of disposable income": 1,\n\t"Personal status and sex": a93,\n\t"Other debtors / guarantors": a101,\n\t"Present residence since": 1,\n\t"Property": a121,\n\t"Age in years": 27,\n\t"Other installment plans": a143,\n\t"Housing": a152,\n\t"Number of existing credits at this bank": 1,\n\t"Job": a172,\n\t"Number of people being liable to provide maintenance for": 1,\n\t"Telephone": a191,\n\t"foreign worker": a201,\n\t"status": 0\n}\n\nsample four:\n{\n\t"Status of existing checking account": a11,\n\t"Duration in month": 47,\n\t"Credit history": a32,\n\t"Purpose": a40,\n\t"Credit amount": 10722.0000,\n\t"Savings account/bonds": a61,\n\t"Present employment since": a72,\n\t"Installment rate in percentage of disposable income": 1,\n\t"Personal status and sex": a92,\n\t"Other debtors / guarantors": a101,\n\t"Present residence since": 1,\n\t"Property": a121,\n\t"Age in years": 35,\n\t"Other installment plans": a143,\n\t"Housing": a152,\n\t"Number of existing credits at this bank": 1,\n\t"Job": a172,\n\t"Number of people being liable to provide maintenance for": 1,\n\t"Telephone": a192,\n\t"foreign worker": a201,\n\t"status": 0\n}\n\nsample five:\n{\n\t"Status of existing checking account": a13,\n\t"Duration in month": 42,\n\t"Credit history": a30,\n\t"Purpose": a49,\n\t"Credit amount": 6289.0000,\n\t"Savings account/bonds": a61,\n\t"Present employment since": a72,\n\t"Installment rate in percentage of disposable income": 2,\n\t"Personal status and sex": a91,\n\t"Other debtors / guarantors": a101,\n\t"Present residence since": 1,\n\t"Property": a122,\n\t"Age in years": 33,\n\t"Other installment plans": a143,\n\t"Housing": a152,\n\t"Number of existing credits at this bank": 2,\n\t"Job": a173,\n\t"Number of people being liable to provide maintenance for": 1,\n\t"Telephone": a191,\n\t"foreign worker": a201,\n\t"status": 0\n}\n\nDirectly generate only one new final sample in JSON format that approximates the key patterns observed in the provided samples. Answer:\n', 'role': 'user'}, {'content': '{\n\t"Age in years": 28,\n\t"Foreign worker": a201,\n\t"Housing": a152,\n\t"Installment rate in percentage of disposable income": 1,\n\t"Job": a173,\n\t"Number of existing credits at this bank": 2,\n\t"Number of people being liable to provide maintenance for": 1,\n\t"Other debtors / guarantors": a101,\n\t"Other installment plans": a142,\n\t"Personal status and sex": a92,\n\t"Present employment since": a72,\n\t"Present residence since": 1,\n\t"Property": a121,\n\t"Purpose": a40,\n\t"Status of existing checking account": a12,\n\t"Status": 0,\n\t"Telephone": a191,\n\t"Total credit": 10722.0000,\n\t"Age": 28,\n\t"Credit history": a30,\n\t"Credit amount": 7515.0000,\n\t"Duration in month": 48,\n\t"Foreign worker": a201,\n\t"Savings account/bonds": a61\n} ', 'role': 'assistant'}]

[{'content': 'Here are five entries of tabular data in JSON format, each consisting of twenty-one features. Each feature is described in a structured JSON format: "feature name": "value" . The target feature status is a classification task.\n\nsample one:\n{\n\t"Status of existing checking account": a14,\n\t"Duration in month": 48,\n\t"Credit history": a31,\n\t"Purpose": a49,\n\t"Credit amount": 3609.0000,\n\t"Savings account/bonds": a61,\n\t"Present employment since": a73,\n\t"Installment rate in percentage of disposable income": 1,\n\t"Personal status and sex": a92,\n\t"Other debtors / guarantors": a101,\n\t"Present residence since": 1,\n\t"Property": a121,\n\t"Age in years": 27,\n\t"Other installment plans": a142,\n\t"Housing": a152,\n\t"Number of existing credits at this bank": 1,\n\t"Job": a173,\n\t"Number of people being liable to provide maintenance for": 1,\n\t"Telephone": a191,\n\t"foreign worker": a201,\n\t"status": 0\n}\n\nsample two:\n{\n\t"Status of existing checking accou

In [13]:
[ { "content": "Part 1. Definition\nYou are given a math word problem and you are supposed to apply division mathematical operator on the numbers embedded in the text to answer the following question and then only report the final numerical answer.\nPart 2. Example\nContext: Alex wants to split a collection of eggs into 4 groups. Alex has 8 eggs and 6 marbles. \n\n Question: How many eggs will each group have?\nAnswer: 2\nExplanation: Total eggs each group will get is 8/4 = 2\nPart 3. Exercise\nContext: 6 packs of dvds can be bought with 120 dollars. \n\n Question: How much does each pack cost?\nAnswer:", "role": "user" }, { "content": "Your task is to define the mathematical operation to be performed on the given numbers to solve the problem and provide a formula to calculate the answer. In this case, the task involves dividing the total amount of eggs by the number of groups, and your answer will be the number of eggs each group will have. I will provide you with the formula to solve the exercise, and if you have any questions, feel free to ask.", "role": "assistant" } ]

[{'content': 'Part 1. Definition\nYou are given a math word problem and you are supposed to apply division mathematical operator on the numbers embedded in the text to answer the following question and then only report the final numerical answer.\nPart 2. Example\nContext: Alex wants to split a collection of eggs into 4 groups. Alex has 8 eggs and 6 marbles. \n\n Question: How many eggs will each group have?\nAnswer: 2\nExplanation: Total eggs each group will get is 8/4 = 2\nPart 3. Exercise\nContext: 6 packs of dvds can be bought with 120 dollars. \n\n Question: How much does each pack cost?\nAnswer:',
  'role': 'user'},
 {'content': 'Your task is to define the mathematical operation to be performed on the given numbers to solve the problem and provide a formula to calculate the answer. In this case, the task involves dividing the total amount of eggs by the number of groups, and your answer will be the number of eggs each group will have. I will provide you with the formula to solve 